In [ ]:
import numpy as np
import random

In [ ]:
# Definimos funciones primitivas para la PG
def add(x, y): return x + y
def sub(x, y): return x - y
def mul(x, y): return x * y
def div(x, y): return x / y if y != 0 else 1

# Generamos una población inicial de árboles de fórmulas
def generar_poblacion(tamano):
    funciones = [add, sub, mul, div]
    terminales = ["peso", "granos", 1, 2, 3]
    poblacion = []
    for _ in range(tamano):
        arbol = [random.choice(funciones), random.choice(terminales), random.choice(terminales)]
        poblacion.append(arbol)
    return poblacion

# Evaluamos una fórmula en un contexto
def evaluar(arbol, contexto):
    if callable(arbol[0]):  # Es una función
        return arbol[0](evaluar(arbol[1], contexto), evaluar(arbol[2], contexto))
    else:  # Es un terminal
        return contexto[arbol[0]] if arbol[0] in contexto else arbol[0]

# Fitness: qué tan bien una fórmula optimiza la velocidad
def calcular_fitness(arbol, datos):
    error_total = 0
    for dato in datos:
        prediccion = evaluar(arbol, dato)
        error_total += abs(prediccion - dato["velocidad_ideal"])
    return 1 / (1 + error_total)

# Ejemplo de datos de entrenamiento
datos_entrenamiento = [
    {"peso": 10, "granos": 5, "velocidad_ideal": 7},
    {"peso": 20, "granos": 10, "velocidad_ideal": 15},
    # Agrega más datos aquí
]

# Evolucionamos la población
def evolucionar(poblacion, datos, generaciones=10):
    for _ in range(generaciones):
        fitness = [calcular_fitness(arbol, datos) for arbol in poblacion]
        seleccionados = random.choices(poblacion, weights=fitness, k=len(poblacion))
        poblacion = []
        for i in range(0, len(seleccionados), 2):
            # Cruzamiento
            padre1, padre2 = seleccionados[i], seleccionados[i+1]
            hijo1 = [padre1[0], padre2[1], padre1[2]]
            hijo2 = [padre2[0], padre1[1], padre2[2]]
            poblacion += [hijo1, hijo2]
        # Mutación
        for arbol in poblacion:
            if random.random() < 0.1:  # Probabilidad de mutar
                arbol[random.randint(0, 2)] = random.choice(["peso", "granos", 1, 2, 3])
    return max(poblacion, key=lambda a: calcular_fitness(a, datos))

# Ejecución
poblacion_inicial = generar_poblacion(10)
mejor_solucion = evolucionar(poblacion_inicial, datos_entrenamiento)
print("Mejor fórmula encontrada:", mejor_solucion)
